[View in Colaboratory](https://colab.research.google.com/github/joocheolkim/IntroToQRM/blob/master/20180630.ipynb)

# Linear Regression

In [5]:
%doctest_mode



Exception reporting mode: Context
Doctest mode is: OFF


In [0]:
>>> from sklearn import svm
>>> clf = svm.SVC(gamma=0.001, C=100.)

In [7]:
from keras.models import Sequential

Using TensorFlow backend.


In [8]:
'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

606208/600901 [==============================] - 0s 0us/step
corpus length: 600893
total chars: 57
nb sequences: 200285
Vectorization...
Build model...
Epoch 1/60
 40448/200285 [=====>........................] - ETA: 2:24 - loss: 3.3080

200285/200285 [==============================] - 179s 893us/step - loss: 3.2831

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: " other equally privileged ones; as soon "
 other equally privileged ones; as soon ee e he  he    eh eee e h   hrrer 

e   h e eheeeh h r hhhhhh h h  e  eehhehe      h eeh e h eeorehe e he    ee hee ee  eh  hh h     eeeheehe eehre h ee o e      hhhhe    hhehe hhhe ee  eh e  eheh r heiheehe eeheheoe  ehhe  h eh e  heh eh   eete   h hhhh he      e   h   e    hen e he    h ehee     e e eeh    h hhe ehhehhehh  e h    ee ereeh      eh  hee   h hhe hh hh e heheeh h hh hhhh  he h h e ehr
----- diversity: 0.5
----- Generating with seed: " other equally privileged ones; as soon "
 other equally privileged ones; as soon bih  i
"r  eiy  oe    hh hre eeh ,nh 
hee ere ehee eehrttorie

 hre  he eeo  h   enhhr  h  reie eherh eoo     lr    h elh  e  heer rr ee e  e t oeeheeh,ethe  eioh eneho reh iheehhhho  teehhnhto e  eh erre ehe  eheeri   m  hn eh eeinhei   t hue,hhhte uehesrht  neeieh inerrhrheeehe ehseeeeieble e   eoh eo ce  ee  i "  n   rh r e ehiheehh  toeeho eeeh ehnee ehtereetrt  er  n hheo h hhhh rehr  eeeheh ii
----- diversity: 1.0
----- Generating with seed: " other equally privileged ones; as soon "
 other equally privileged ones; as soon  ,hysoghy,r
rhnlnrdeonsirehoueioet ei "yrsnoe elhne
rlmermho
hrrir 
 rri
hhhehooo

sft
o orrg sislh,t eseic s, hhothhonironnterhco ieh  rhsd "iut ei  o thny2r o
e rhrrii. rh,h nneot lr h"hihvcirteh tnno
heeeilitier et iuow i
nlohphu r ryhhyi rl "he hiieh olee r    tea
   ue hee  "hr eerhiysh yoehen"ehsuhc
n,olmn vrob eeir e" ox n iro iar"h ter  tdix nihmuht"yoe   hrh"inlhahnhsih
snhr
 e eikorenhs

 
----- diversity: 1.2
----- Generating with seed: " other equally privileged ones; as soon "
 other equally privileged ones; as soon ntee,errhbirre"e  neenhohr  d ier dehiyhteh,i,btoio? reeehxr derenneihlt ea,igstee r h"eirchnerb

...

200285/200285 [==============================] - 180s 898us/step - loss: 3.2875

----- Generating text after Epoch: 59
----- diversity: 0.2
----- Generating with seed: " of the blue, wanton sea and the mediter"
 of the blue, wanton sea and the mediterfaaohaaataaaafaiaiahiooiaaihaa

aiiaaaaaaaaaiihaiaaaaiiiaaiaaaiafaahaaafaaaaaahaaaainaaahi-aiaaiafiaaaaafaaaahaaiiaaiiaiaaaaaaiaaaaia-ai-iih-aiaiaaaiaaaaahahaaiaoaiahaaaiaaaaaaifaaaahiiaaaoiaafaaaaaaaaiaiihaaaaiaahafaaaaaaiiaahaaiaaaafaaaaaaaaaiiaaaaahaiiaaaaihaaaafaaiaiahiiiaaiaiafhfiaaaaaaaiaiaaiiaiaaiiiaaaiiiaaahaaaiaiaiaaiaaohfaiaaiaaiaoaiidaaiiaiaaahaaaafhaaafaoaaiiah-iaaafaaooaahaaiaaaihiiaaaa
----- diversity: 0.5
----- Generating with seed: " of the blue, wanton sea and the mediter"
 of the blue, wanton sea and the mediterhfiafiihfhoiiifa-aaieihoao ium-ahofooiaaaaiiaae-saeeiihfi

iohooalnafahata hhtchehia-ifhiafagihic hodhiahie fofaaiao-of-aaaa-srisaan-iafntitiaiaioanoaoiahiicioa-hhaihihhhffofaoahofhafthiiahiafaiaoiihfu-aoaoaacha-ian haaiha t-i-ffiafiihfiaio haieiaaiauhiaai-ocaaa-ifhaiata-aautno aiiiniahtasfc faaiaaffaaafiaao aaaaia-acfeieaasififhctahfnccohhhaaafhaasoeowaf iiifhoaiacaacthhoa-acaaetaioaogacaafaai -oia
----- diversity: 1.0
----- Generating with seed: " of the blue, wanton sea and the mediter"
 of the blue, wanton sea and the meditercuieffanohaa-a-n iataf,agoweaiffatf aeh?hlhehwfnlaafa h-adny.ihawhheox-fuoaodii

oua  nafaniana-annhlhtice-caeoaifaf-tt elctp-oaiaffd-a-otaalanabfu-homhishoagoithasd ma-asoaiag ua-fddaiifmqfaeedsuif- ioin-soalhia
uh-irihuis
dnaanauuoaf h hoedoeiycdaos-hahheeutfd-deniifafioasfn tfcpfnffhhhanhih?oh-cadcicns xiidtaifh.-h-hoaaohhddialufoiglaofiet aiffacfstdoemageici-oih .veolahtunsootahgaatn pnfi.hhhufa
----- diversity: 1.2
----- Generating with seed: " of the blue, wanton sea and the mediter"
 of the blue, wanton sea and the mediterthghg.-ibffct-hynihlaasiidaohulpttgo  athfoelificghhdyhhaofdu-nhuocani aifnui-ignf haaeo-sif nr

-ndi-a-ohpiideistuaaulcho ahuydng
ghhfceewcxhbdhh.aa-ifuhefhgitmtathtund;aataoont?saahelfthnfxaeefehme-u.dvaanggoa-cihuiav-iaflfnnuaaasstrnul?cltbdd -f-amaawvgad-fif
id?oht-arfuhaflcfng ul-h"uhiht--sg wgfaicf-ta 
iuardwtphh-aswlgncfafoeiitas.fp-t siaia" uolh- dtuoi  adiahdaoye-i-
if
wnetaiusehwtgm ts:thy
